In [1]:
import tensorflow as tf
import numpy as np
#import tfgraphviz as tfg
from tensorboard import notebook

In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [3]:
reset_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [5]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

model = tf.keras.Sequential()

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(inputs=X, units=n_hidden1, activation=tf.nn.relu)
    hidden2 = tf.layers.dense(inputs=hidden1, units=n_hidden2, activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=hidden2, units=n_outputs)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
n_epochs = 40
batch_size = 50
min_error = float("inf")
c_error = 0

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        error = loss.eval(feed_dict={X: X_valid, y: y_valid})
        
        if(error < min_error):
            save_path = saver.save(sess, "./my_model_finalv1.ckpt")
            min_error = error
            c_error = 0
        else:
            c_error = c_error + 1
            
        if(c_error == 50 or min_error <1e-9):
            break
        
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test, "Test error: ", error)

0 Train accuracy: 0.8996182 Test accuracy: 0.9028 Test error:  0.3538575
1 Train accuracy: 0.9196 Test accuracy: 0.9252 Test error:  0.27321905
2 Train accuracy: 0.93029094 Test accuracy: 0.9372 Test error:  0.23675342
3 Train accuracy: 0.9381273 Test accuracy: 0.942 Test error:  0.21015103
4 Train accuracy: 0.94330907 Test accuracy: 0.9472 Test error:  0.19060422
5 Train accuracy: 0.9471273 Test accuracy: 0.9512 Test error:  0.17913444
6 Train accuracy: 0.9525273 Test accuracy: 0.9548 Test error:  0.16226868
7 Train accuracy: 0.9577091 Test accuracy: 0.9612 Test error:  0.15049107
8 Train accuracy: 0.9606182 Test accuracy: 0.962 Test error:  0.14204995
9 Train accuracy: 0.96334547 Test accuracy: 0.965 Test error:  0.1355148
10 Train accuracy: 0.9665273 Test accuracy: 0.9654 Test error:  0.12784617
11 Train accuracy: 0.968 Test accuracy: 0.9672 Test error:  0.121148646
12 Train accuracy: 0.96918184 Test accuracy: 0.9682 Test error:  0.12020469
13 Train accuracy: 0.9717636 Test accuracy

In [7]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv1.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_finalv1.ckpt


In [8]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
